## Assessment 2 - Building LR Model for Delhi Temprature Dataset

## importing required libraries 

In [131]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error


In [132]:
df=pd.read_csv('F:\Dataset\Delhi Temerature.csv')          ## reading dataset presnet in local device
df

,datetime_utc,_conds,_dewptm,_fog,_hail,_heatindexm,_hum,_precipm,_pressurem,_rain,_snow,_tempm,_thunder,_tornado,_vism,_wdird,_wdire,_wgustm,_windchillm,_wspdm
0,19961101-11:00,Smoke,9.0,0,0,NaN,27.0,NaN,1010.0,0,0,30.0,0,0,5.0,280.0,West,NaN,NaN,7.4
1,19961101-12:00,Smoke,10.0,0,0,NaN,32.0,NaN,-9999.0,0,0,28.0,0,0,NaN,0.0,North,NaN,NaN,NaN
2,19961101-13:00,Smoke,11.0,0,0,NaN,44.0,NaN,-9999.0,0,0,24.0,0,0,NaN,0.0,North,NaN,NaN,NaN
3,19961101-14:00,Smoke,10.0,0,0,NaN,41.0,NaN,1010.0,0,0,24.0,0,0,2.0,0.0,North,NaN,NaN,NaN
4,19961101-16:00,Smoke,11.0,0,0,NaN,47.0,NaN,1011.0,0,0,23.0,0,0,1.2,0.0,North,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100985,20170424-06:00,Haze,17.0,0,0,NaN,25.0,NaN,1005.0,0,0,34.0,0,0,4.0,320.0,NW,NaN,NaN,11.1
100986,20170424-09:00,Haze,14.0,0,0,NaN,16.0,NaN,1003.0,0,0,38.0,0,0,4.0,320.0,NW,NaN,NaN,22.2
100987,20170424-12:00,Haze,12.0,0,0,NaN,14.0,NaN,1002.0,0,0,36.0,0,0,4.0,270.0,West,NaN,NaN,18.5
100988,20170424-15:00,Haze,15.0,0,0,NaN,27.0,NaN,1004.0,0,0,32.0,0,0,2.0,320.0,NW,NaN,NaN,3.7


In [133]:
print("Total rows and column of our main dataset-->",df.shape)

Total rows and column of our main dataset--> (100990, 20)


## droping null values

In [134]:
null_data = df.isnull().sum()/df.shape[0]*100
null_columns = null_data[null_data>20].keys()        # fetching columns having more than 20% of missing data
df = df.drop("datetime_utc",axis=1)                  # no need of this column
print("Droping those columns have large amout of null values\n\n",null_columns)

Droping those columns have large amout of null values

 Index([' _heatindexm', ' _precipm', ' _wgustm', ' _windchillm'], dtype='object')


In [135]:
df = df.drop(columns=null_columns)                         #droping columns 
print("Total rows and column of our main dataset after removing null columns-->",df.shape)              

Total rows and column of our main dataset after removing null columns--> (100990, 15)


## finding missing numerical null data

In [136]:
X_num = df.select_dtypes(include=["int64","float64"])                                         # select numerical data type
print("Number of null data present in our Numerial dataset",X_num.isnull().sum().sum())

Number of null data present in our Numerial dataset 23824


In [137]:
num_nn = [var for var in X_num.columns if X_num[var].isnull().sum()>0]      #finding numerical columns having missing values
print("Numerical data having null values\n\n",num_nn)

Numerical data having null values

 [' _dewptm', ' _hum', ' _pressurem', ' _tempm', ' _vism', ' _wdird', ' _wspdm']


## finding missing object null data

In [138]:
X_char = df.select_dtypes(include=["object"])                                # selecting stirng data
char_nn = [var for var in X_char.columns if X_char[var].isnull().sum()>0]
print("Character data having null values\n\n",char_nn)

Character data having null values

 [' _conds', ' _wdire']


## filling values

In [139]:
num_nn = [' _dewptm', ' _hum', ' _pressurem', ' _vism', ' _wdird', ' _wspdm']      #missing numerical data col
char_nn = [' _conds', ' _wdire']                                                   # missing char data col

#creating two pipeline one is median( for numerical missing data) and other is mode(for string missing data)
meadian_impute = Pipeline(steps=[("imputer",SimpleImputer(strategy="median"))])   
mode_impute = Pipeline(steps=[("imputer",SimpleImputer(strategy="most_frequent"))])

In [140]:
transformer = ColumnTransformer(transformers=[("median",mean_impute,num_nn),          #applying strategy
                                             ("mode",mode_impute,char_nn)
                                             ])

In [141]:
transformer.fit(df)                                                                        


ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
                  transformer_weights=None,
                  transformers=[('median',
                                 Pipeline(memory=None,
                                          steps=[('imputer',
                                                  SimpleImputer(add_indicator=False,
                                                                copy=True,
                                                                fill_value=None,
                                                                missing_values=nan,
                                                                strategy='mean',
                                                                verbose=0))],
                                          verbose=False),
                                 [' _dewptm', ' _hum', ' _pressurem', ' _vism',
                                  ' _wdird', ' _wspdm']),
                                ('mode',
     

In [142]:
#applying these changes in our main dataset(filling missing values)

updated_values = transformer.transform(df)
update_df = pd.DataFrame(updated_values,columns=num_nn+char_nn)                     
df.update(update_df)
df.isnull().sum()

 _conds          0
 _dewptm         0
 _fog            0
 _hail           0
 _hum            0
 _pressurem      0
 _rain           0
 _snow           0
 _tempm        673
 _thunder        0
 _tornado        0
 _vism           0
 _wdird          0
 _wdire          0
 _wspdm          0
dtype: int64

In [143]:
df = df.dropna(subset=[" _tempm"])            #droping N/A values form this column because it has 637 null values
df.isnull().sum().sum()

0

In [144]:
print("Total null values in our dataset -->",df.isnull().sum().sum())

Total null values in our dataset --> 0


## dividing dataset

In [145]:
X = df.drop(columns=[" _tempm"])                        #we have to predict the _tempm column by Linear regression 
y = df[" _tempm"]

## label encoding

In [146]:
enc = LabelEncoder()
end = np.array([enc.fit_transform(X[var]) for var in char_nn])
X[' _conds'] = end[0]                                                             
X[' _wdire'] = end[1]

## feature scaling

In [147]:
sc = StandardScaler()
sc.fit(X)
X = sc.transform(X)                             #applying standard scaler scaling 

## splitting dataset in test and train

In [148]:
X_train,X_test, Y_train,Y_test = train_test_split(X,y,test_size=.3)    

## preparing model

In [149]:
model = LinearRegression()
model = model.fit(X_train,Y_train)

In [150]:
print(round(model.score(X_test,Y_test)*100))                  #accuracy

92.0


In [151]:
Y_pred = model.predict(X_test)


In [152]:
mse = mean_squared_error(Y_test,Y_pred)
mse                                                        #mean squared error

6.09805086925765

In [153]:
rmse=np.sqrt(mse)
rmse                                                         #root mean squared error

2.469423185534964